In [ ]:
# Check if empty lists makes difference.
# Perhaps remove section where it references to wordnet.

In [2]:
#######################
# Accept search term from user and download
# the last 100 tweets from that search term
#######################

# Install the python-twitter module. Provides python like interface to the Twitter API. 
# Twitter API is a REST API, it provides information in the form of a JSON which
# will need to be parsed- python-twitter will do this.

import twitter

# Call pydoc twitter.api at the anaconda terminal to access documentation. 

# Twitter API keys are needed to access tweets - both from feature vector and 
# tweets for classification.

api = twitter.Api(consumer_key='6b9ebwaNU4DDa9G9xF1FhrZQt',
                 consumer_secret='2prgddykMj2b7b9zTeN78BBhdrgdaNxjtSyyoo8iNRzKAZhzMX',
                 access_token_key='817521154847969280-bc6J796tc0cRjlhigiRZIoQVIzeW2Hf',
                 access_token_secret='WAZy2gZ9Ok8NdP3W8TOMNliSUUGrLjesudvqA3nEEh9wH')

print api.VerifyCredentials()

{"created_at": "Sat Jan 07 00:00:06 +0000 2017", "default_profile": true, "default_profile_image": true, "friends_count": 101, "id": 817521154847969280, "id_str": "817521154847969280", "lang": "en", "name": "Josh Jones", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "Jonesy_259"}


In [4]:
# Function accepts search term and then fetches the tweets for that term
def createTestData(search_string):
    try:
        tweets_fetched=api.GetSearch(search_string, count=100)
        # This will return a list with twitter.Status objects. These include
        # text, hashtags etc of the tweets that are fetched. 
        print "num tweets: "+str(len(tweets_fetched))+" term: "+search_string
        # Since these tweets don't have sentiment labels yet we will 
        # keep the label empty
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print "Error"
        return None
    
search_string=input("Search for: ")
testData=createTestData(search_string)
    

Search for: '@realDonaldTrump'
num tweets: 100 term: @realDonaldTrump


In [3]:
testData[0:20]

[{'label': None,
  'text': u'Today the media hyperventilated when @realDonaldTrump teased the monthly Jobs Report.\nThey dared us to find a presi\u2026 https://t.co/JD2cZd7Ujr'},
 {'label': None,
  'text': u'.@realDonaldTrump is smiling next to a man who runs a gulag jailing some 200,000 North Koreans and who oversaw the\u2026 https://t.co/FI5VCSqk4o'},
 {'label': None,
  'text': u'In just two years, @realDonaldTrump has done more for the black community than any President in modern history, inc\u2026 https://t.co/yDWRfaHVzI'},
 {'label': None,
  'text': u'RT @realDonaldTrump: Real @FoxNews is doing great, Fake News CNN is dead! https://t.co/1p37tPiB3v'},
 {'label': None,
  'text': u'RT @oldsourpuss: @jbmartinjbmart @steve_vladeck @realDonaldTrump They\u2019re claiming Trump couldn\u2019t have obstructed justice because he didn\u2019t\u2026'},
 {'label': None,
  'text': u"RT @Real_Gaz: John Brennan says @realDonaldTrump is paranoid and insecure\n\nWell if that ain't the pot calling the

In [135]:
#######################
# Create Training Corpus
#######################

# Using Sentiment 140 Training Corpus, has 1,600,000 classified tweets - based
# on whether the tweet used a :) or :( emoticon
def createTrainingCorpus(corpusFile):
    import csv
    corpus=[]
    with open(corpusFile, 'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineReader:
            corpus.append({"text":row[5],"label":int(row[0])})
    
    # Only 20,000 positive and negative tweets respectively. Processing
    # 1,600,000 tweets will take too long, and possibly introduce overfitting
    trainingData=[]
    for label in [0,4]:
        i=1
        for tweet in corpus:
            if tweet["label"]==label and i<=10000:
                trainingData.append(tweet)
                i+=1
    return trainingData

corpusFile="/Users/Josh/Desktop/Software Project/trainingCorpus/trainingData1600000.csv"
trainingData=createTrainingCorpus(corpusFile)

In [144]:
# Process tweets, remove non-dictionary words, punctuation, links etc.
import re
from timeit import default_timer as timer
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['rt', "'s", 'i'])
        
    def processTweets(self, list_of_tweets):
        # The list of tweets is a list of dictionaries which has the keys, "text" and "label"
        processedTweets=[]
        # Each tuple is a list of words + label.
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        # Convert to lowercase
        tweet=tweet.lower()
        # RemoveLinks
        tweet=re.sub('https?://[A-Za-z0-9./]+','',tweet)
        # Remove '@' 
        tweet=re.sub(r'@[A-Za-z0-9]+','',tweet)
        # Replace #word with word
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        # Remove non-letters
        tweet=re.sub("[^a-zA-Z]", " ",tweet)
        
        tweet=word_tokenize(tweet)
        # Converts tweet to list of words
        # Remove stopwords
        tweet = [word for word in tweet if word not in self._stopwords]
        # If word is not in wordnet, remove it.
        for index, word in enumerate(tweet):
            if len(word) < 2:
                tweet[index] = 'i'
            var = wn.synsets(word)[:1]
            if len(var) < 1:
                tweet[index] = 'i'
        return [word for word in tweet if word not in self._stopwords]
        

tweetProcessor=PreProcessTweets()
start = timer()
ppTrainingData=tweetProcessor.processTweets(trainingData)
end = timer()
print (end - start)
ppTestData=tweetProcessor.processTweets(testData)  

6.86916327204


In [147]:
# Store processed tweets in CSV, as  processing tweets takes considerable
# time, this will mean we only have to do it once.

def storePPTrainingData(ppTrainingData,tweetDataFile):
    import csv
    with open(tweetDataFile,'wb') as csvfile:
        linewriter=csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in ppTrainingData:
            try: 
                linewriter.writerow([tweet[0],tweet[1]])
            except Exception,e:
                print e
                
tweetDataFile="/Users/Josh/Desktop/Software Project/trainingCorpus/processedCorpus.csv"
storePPTrainingData(ppTrainingData,tweetDataFile)

In [ ]:
# Extract features and train classifier, two methods: Naive Bayes + SVM

import nltk
# Naive Bayes Classifier -Using NLTK's built in classifier

# Build Vocabulary
def buildVocabulary(ppTrainingData):
    all_words=[]
    for (words,sentiment) in ppTrainingData:
        all_words.extend(words)
        # Words are not duped, all words present in corpus, correlates 
        # to how many times it appears in corpus
        wordlist=nltk.FreqDist(all_words)
        # Create dictionary for all words with frequency
        word_features=wordlist.keys()
        # Return unique list of words in corpus, just the keys from
        # above dictionary
        return word_features

# NLTK has an apply_features function that takes a user defined
# function to extract features from training data. The following extract feature
# function takes each tweet and represents it with the presence or absence
# of words in our vocabulary (every word in training data/corpus)

def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
        # This will make a disctionary with keys like 'contains word1
        # etc, with values True or False
    return features

# Extract features and train classifier
word_features = buildVocabulary(ppTrainingData)
trainingFeatures=nltk.classify.apply_features(extract_features,ppTrainingData)
# apply_features will take the function extract_features function defined above, 
# and aply it to each element of ppTrainingData. It automatically identifies that 
# each of those elements (in ppTrain...) is actually a tuple, so itt takes the first element
# of the tuple to be text and the second element to be the label, and applies
# the function only to the text

NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)
# We now have classifier that has been trained using NaiveBayes, NLTK library 

# Support Vector Machine - here we use sentiwordnet to weight words
# based on their usage in postive/negative conexts
# from nltk.corpus import sentiwordnet as swn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Data form will have to be changed slightly. SKLearn has a CountVectorizer object. 
# This will take in documents and directly return a term-document matrix with
# the frequencies of a word in the document. It builds the vocabulary by itself (takes a tweet 
# + returns terms with their frequencies). TrainingData and the labels (sentiment) will
# be given to the SVM classifier seperately and not as tuples. Another distinguishment
# between Naive Bayes is that it is a binary classifier- can only classify data 
# into two classes, no 'neutral'

svmTrainingData=[' '.join(tweet[0]) for tweet in ppTrainingData]
# Create sentences out of the lists of words

vectorizer=CountVectorizer(min_df=1)
X=vectorizer.fit_transform(svmTrainingData).toarray()
# X is the term document matrix
vocabulary=vectorizer.get_feature_names()

# Use sentiwordnet to add weights to features.

swn_weights=[]

for word in vocabulary:
    try:
        # Try block as not all words in trainingData will be in sentiwordnet. Look
        # for synsets of that word in sentiwordnet
        synset=list(swn.senti_synsets(word))
        # use the first synset only to compute score, as this will represent
        # the most common (meaning) usage of that word
        common_meaning=synset[0]
        # If the positive score is greater, use that as the weight, vice versa. 
        if common_meaning.pos_score()>common_meaning.neg_score():
            weight=common_meaning.pos_score()
        elif common_meaning.pos_score()<common_meaning.neg_score():
            weight=-common_meaning.neg_score()
        else:
            weight=0
    except:
        weight=0
    swn_weights.append(weight)
    
# Multiply each array in original matrix with the following weights
# Initialise a list

swn_X=[]
for row in X:
    swn_X.append(np.multiply(row,np.array(swn_weights)))
# Convert the list to a numPy array
swn_X=np.vstack(swn_X)

# Represent labels as positive=1, negative=2 so that everything is respresented numPy arrays.
#labels_to_array={"positive":1,"negative":2}
#labels=[labels_to_array[tweet[1]] for tweet in ppTrainingData]
labels=[tweet[1] for tweet in ppTrainingData]
y=np.array(labels)

# Build SVM Classifier
from sklearn.svm import SVC
SVMClassifier=SVC()
SVMClassifier.fit(swn_X,y)

In [41]:
# Run the classifier on downloaded tweets

# NaiveBayes
NBResultLabels=[NBayesClassifier.classify(extract_features(tweet[0])) for tweet in ppTestData]

# SVM
SVMResultLabels=[]
for tweet in ppTestData:
    tweet_sentence=' '.join(tweet[0])
    svmFeatures=np.multiply(vectorizer.transform([tweet_sentence]).toarray(),np.array(swn_weights))
    SVMResultLabels.append(SVMClassifier.predict(svmFeatures)[0])
    # Append 1 or 2 to the list, as tweet is postive/negative

In [47]:
# Get majority vote + return sentiment

# NaiveBayes
if NBResultLabels.count('positive')>NBResultLabels.count('negative'):
    print "NB Result Positive Sentiment: " + str(100*NBResultLabels.count('positive')/len(NBResultLabels))
else:
     print "NB Result Negative Sentiment: " + str(100*NBResultLabels.count('negative')/len(NBResultLabels))
        
# SVM
if SVMResultLabels.count(1)>SVMResultLabels.count(2):
    print "SVM Result Positive Sentiment: " + str(100*SVMResultLabels.count(1)/len(SVMResultLabels))
else:
     print "SVM Result Negative Sentiment: " + str(100*SVMResultLabels.count(2)/len(SVMResultLabels))

NB Result Positive Sentiment: 100
SVM Result Positive Sentiment: 100
